## 用numpy写一个神经网络实现MINST

In [147]:
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import ToTensor

利用pytorch下载的数据集，稍后会转换成ndarray给numpy使用

In [148]:
# 指定函数的返回类型，方便IDE进行提示
# def get_data() -> tuple[np.ndarray, np.ndarray]:
# 不写返回类型也行，一般函数写的越简单越好
def get_data():
    training_data = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor()
    )
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor()
    )
    tr = training_data.data / 255
    te = training_data.targets
    return tr.numpy(), te.numpy()

初始化网络，输入是应该是28*28=784 
第一层输出50， 第二层输出100
最后一层即第三层输出10个数，代表结果，输出函数可以用softmax

In [149]:
def init_network(size_x):
    nn = {
        'w1': np.random.randn(size_x, 50),
        #这里用rand(50)也行，加法的广播机制
        'b1': np.random.randn(1, 50), 

        'w2': np.random.randn(50, 100),
        'b2': np.random.randn(1, 100),

        'w3': np.random.randn(100, 10),
        'b3': np.random.randn(1, 10),
    }
    return nn


激活函数，输出函数

In [150]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    c = np.max(x)
    exp_a = np.exp(x - c) # 减去c是为了防止溢出
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

正向运行网络，推理

In [151]:
def forward(nn, x):
    w1, w2, w3 = nn['w1'], nn['w2'], nn['w3'] 
    b1, b2, b3 = nn['b1'], nn['b2'], nn['b3']
    
    a1 = x.dot(w1) + b1
    y1 = sigmoid(a1)
    
    a2 = y1.dot(w2) + b2
    y2 = sigmoid(a2)
    
    a3 = y2.dot(w3) + b3
    y3 = softmax(a3)
    
    return y3


运行一次网络

In [152]:
data, labels = get_data()
i = np.random.randint(60000)
x = data[i].reshape(1, 28 * 28)
t_index = labels[i]
nn = init_network(x.size)
y = forward(nn, x)
y

array([[0.  , 0.  , 0.  , 0.12, 0.  , 0.83, 0.  , 0.  , 0.05, 0.  ]])

接下来，定义两种损失函数, 这里增加了批量size

In [205]:
# 均方误差 输出与目标的差的平方和，除以2
def mean_square_error(y, t):
    batch_size = y.shape[0]
    return 0.5 * np.sum((y - t) ** 2) / batch_size

# 交叉熵误差: 目标*loge(y)的和的负数 这个经常与sigmoid激活函数同时使用
def cross_entropy_error(y, t):
    # 由于loge(0)是负无穷大-inf，计算机无法继续之后的运算
    # 所以给输入增加一个微小的数，并且不影响结果
    delta = 1e-7
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + delta)) / batch_size


看一下两种损失函数

In [224]:
t = np.zeros(10)
t[int(t_index)] = 1



e1 = mean_square_error(y, t)
e2 = cross_entropy_error(y, y)

# 设置np的打印选项：精度最大为小数点后2位，不用科学计数法
np.set_printoptions(precision=2, suppress=True)

print('t =', t)
print('y =', y)
print(f'e1 = {e1:5f}')
print(f'e2 = {e2:5f}')


t = [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y = [[0.   0.   0.   0.12 0.   0.83 0.   0.   0.05 0.  ]]
e1 = 0.802181
e2 = 0.583236


接下来看一看损失函数的批量处理是否正确 每一个损失函数的维度为10，总共100个

In [226]:
# 生成一个1000 * 10 的targets，每一列的元素是10个，其中一个为1，其余为0，与上边手写一样
t_test = np.zeros((1000, 10))
r_indices = np.arange(1000)
c_indices = np.random.choice(10, 1000)
c_indices
t_test[r_indices, c_indices] = 1

# y也是1000 * 10， 每一列的输出经过softmax处理，更真实了，
# 这里softmax没有做批量处理，所以用了循环
y_test = np.random.normal(0, 1, (1000, 10))
r_t_indices = np.arange(1000)

for i in range(1000):
    y_test[i] = softmax(y_test[i])
    
e1 = mean_square_error(y_test, t_test)
print('batch_test e1:', e1)
e2 = cross_entropy_error(y_test, t_test)
print('batch_test e2:', e2)

batch_test e1: 0.4985116597049505
batch_test e2: 2.746850291489507
